In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import dates
import datetime

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

## Load train/test dataset

In [ ]:
train = pd.read_csv('../input/covid19-global-forecasting-week-1/train.csv')
test = pd.read_csv('../input/covid19-global-forecasting-week-1/test.csv')

## Check the data feature.
Let's check below feature on train/test set
* date period
* country count

In [ ]:
print ("How many Province on train set ==> " +str(len(train['Province/State'].unique())))
print ("How many country on train set ==> " +str(len(train['Country/Region'].unique())))
print ("Date period for train set ==> " +train['Date'].unique()[0]+" to "+train['Date'].unique()[-1])
print ("How many Province on test set ==> " +str(len(test['Province/State'].unique())))
print ("How many country on test set ==> " +str(len(test['Country/Region'].unique())))
print ("Date period for test set ==> " +test['Date'].unique()[0]+" to "+test['Date'].unique()[-1])

### There are time overlap for train/test set ==> 2020-03-12 to 2020-03-18
So we have 03-12 to 03-18's answer on train set, submit will freeze on **2020-03-26**, we need to predict how many comfirmedcase and fatalities on **2020-03-26** to **2020-04-23** time period, LB will keep update until **2020-04-23**

In [ ]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import dates
import datetime

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [ ]:
train = pd.read_csv('../input/covid19-global-forecasting-week-1/train.csv')
test = pd.read_csv('../input/covid19-global-forecasting-week-1/test.csv')
# dictionary = {'*':''}
# train = train.replace(dictionary, regex=True, inplace=True)
# train = train[(train['Country/Region'] == "US") &(train['Province/State'] == "New York")]
# test = test[(test['Country/Region'] == "US") & (test['Province/State'] == "New York")]
us_train = train[train['Country/Region'] == 'US']
us_test = test[test['Country/Region'] == 'US']
# us_train.head()
# us_test.head() 
tw_train = train[train['Country/Region'] == 'Taiwan*']
tw_test = test[test['Country/Region'] == 'Taiwan*']


china_train = train[train['Country/Region'] == 'China']
china_test = test[test['Country/Region'] == 'China']


germany_train = train[train['Country/Region'] == 'Germany']
germany_test = test[test['Country/Region'] == 'Germany']
# np.array(range(len(test))).reshape((-1,1))
countrylist = [[us_train, us_test], [tw_train, tw_test], [germany_train, germany_test], [china_train, china_test]]
# countrylist = [[us_train, us_test]]

In [ ]:
us_test.head(5)

In [ ]:
def run_model(df_list):        
    for country in train['Country/Region'].unique():
        print ('training model for country ==>'+country)
        country_pd_train = train[train['Country/Region']==country]
        country_pd_test = test[test['Country/Region']==country]
#         print(country_pd_train['Province/State'].isna().unique()) 
        if country_pd_train['Province/State'].isna().unique()==True:   
            x = np.array(range(len(country_pd_train))).reshape((-1,1))
            y = country_pd_train['ConfirmedCases']
            model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                             ('linear', LinearRegression(fit_intercept=False))])
            model = model.fit(x, y)
    #         print(len(country_pd_test))
    #         print(range(len(country_pd_test)))
    #         print(np.array(range(len(country_pd_test)))+50)
            predict_x = (np.array(range(len(country_pd_test)))+60).reshape((-1,1))

            test.loc[test['Country/Region']==country,'ConfirmedCases'] = model.predict(predict_x)

        else:
            for state in country_pd_train['Province/State'].unique():
                state_pd = country_pd_train[country_pd_train['Province/State']==state] 
                state_pd_test = country_pd_test[country_pd_test['Province/State']==state] 
                x = np.array(range(len(state_pd))).reshape((-1,1))
                y = state_pd['ConfirmedCases']
                model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                             ('linear', LinearRegression(fit_intercept=False))])
                model = model.fit(x, y)
                predict_x = (np.array(range(len(state_pd_test)))+60).reshape((-1,1))
                test.loc[(test['Country/Region']==country)&(test['Province/State']==state),'ConfirmedCases'] = model.predict(predict_x)
    return(train,test)

In [ ]:
def plot_chart(train, test):
    country = train['Country/Region'].unique()
    print(str(country[0]))
    df_train = train.drop(columns=['Id'])
    df_test = test.drop(columns=['ForecastId'])
    df_train['type'] = 'history'
    df_test['type'] = 'forecast'
    df_all = pd.concat([df_train, df_test])
    grouped = df_all.groupby(['Date',  'type'])['Date', 'ConfirmedCases'].sum().reset_index()
    fig = px.line(grouped, x="Date", y="ConfirmedCases", color = 'type', title= str(country[0]) + " Confirmed Cases Over Time")
    # fig.update_traces(mode='markers+lines')
    fig.show()

In [ ]:
for i in countrylist:
    train = i[0]
    test = i[1]
    train, test= run_model(countrylist)
    plot_chart(train, test)



In [ ]:
for country in train['Country/Region'].unique():
    print ('training model for country ==>'+country)
    country_pd_train = train[train['Country/Region']==country]
    country_pd_test = test[test['Country/Region']==country]
    if country_pd_train['Province/State'].isna().unique()==True:
        x = np.array(range(len(country_pd_train))).reshape((-1,1))
        y = country_pd_train['Fatalities']
        model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                         ('linear', LinearRegression(fit_intercept=False))])
        model = model.fit(x, y)
        predict_x = (np.array(range(len(country_pd_test)))+50).reshape((-1,1))
        test.loc[test['Country/Region']==country,'Fatalities'] = model.predict(predict_x)
    else:
        for state in country_pd_train['Province/State'].unique():
            state_pd = country_pd_train[country_pd_train['Province/State']==state] 
            state_pd_test = country_pd_test[country_pd_test['Province/State']==state] 
            x = np.array(range(len(state_pd))).reshape((-1,1))
            y = state_pd['Fatalities']
            model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                         ('linear', LinearRegression(fit_intercept=False))])
            model = model.fit(x, y)
            predict_x = (np.array(range(len(state_pd_test)))+50).reshape((-1,1))
            test.loc[(test['Country/Region']==country)&(test['Province/State']==state),'Fatalities'] = model.predict(predict_x)

In [ ]:
df_train = train.drop(columns=['Id'])
df_test = test.drop(columns=['ForecastId'])
df_train['type'] = 'history'
df_test['type'] = 'forecast'
df_all = pd.concat([df_train, df_test])

grouped = df_all.groupby(['Date',  'type'])['Date', 'ConfirmedCases'].sum().reset_index()


fig = px.line(grouped, x="Date", y="ConfirmedCases", color = 'type', title="Taiwan Confirmed Cases Over Time")
# fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
df_train = train.drop(columns=['Id'])
df_test = test.drop(columns=['ForecastId'])
df_train['type'] = 'history'
df_test['type'] = 'forecast'
df_all = pd.concat([df_train, df_test])

grouped = df_all.groupby(['Date',  'type'])['Date', 'ConfirmedCases'].sum().reset_index()


fig = px.line(grouped, x="Date", y="ConfirmedCases", color = 'type', title="Taiwan Confirmed Cases Over Time")
# fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
# test[test['Country/Region'] == 'China']
# # test[test[['Country/Region', 'Province/State']].notnull().all(1)]